In [632]:
import random
import math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time

In [895]:
# merge function
def Merge_Backwards(a, b, order):
    c = pd.DataFrame(columns = memory.columns)
    while((len(a)!= 0) and (len(b) != 0)):
        if (b.loc[b.first_valid_index()][order] < a.loc[a.first_valid_index()][order]):
            #add b[0] to the end of c
            c = c.append(b.loc[b.first_valid_index()])
            #remove b[0] from b
            b = b.drop(b.first_valid_index())   
        else:
            # add a[0] to the end of c
            c = c.append(a.loc[a.first_valid_index()])
            #rmeove a[0] from a
            a = a.drop(a.first_valid_index())
 
    while (len(a) != 0):     
        # add a[0] to the end of c
            c = c.append(a.loc[a.first_valid_index()])
            #rmeove a[0] from a
            a = a.drop(a.first_valid_index()) 
    while (len(b) != 0):     
        # add b[0] to the end of c
            c = c.append(b.loc[b.first_valid_index()])
            #rmeove a[0] from a
            b = b.drop(b.first_valid_index())  
    return c

In [896]:
def MergeSort_Backwards(mem, order):
    n = len(mem)
    if ( n == 1 ): return mem
    
    l1 = mem[0:math.floor(n/2)]
    l2 = mem[math.floor(n/2) : n]

    l1 = MergeSort_Backwards( l1, order )
    l2 = MergeSort_Backwards( l2, order )

    return Merge_Backwards( l1, l2, order )

In [897]:
#getNextMinAddress returns the next min address possible for the specified entry i in the current Align List
def getNextStartAddress(AlignList, i):
    if(i==0):
        #assign address to min TLB value, for now use 0
        StartAddress = AlignList.iloc[i-1]['Start Address']
    else:
        min_address = (AlignList.iloc[i-1]['Start Address'] + AlignList.iloc[i-1]['Size']+1)
        alignment = AlignList.iloc[i]['Alignment']
        StartAddress = (min_address + alignment) - (min_address + alignment)%alignment
    return StartAddress     
    

In [898]:
#getNextMinAddress returns the next min address possible for the specified entry i in the current Align List
def getNextStartAddress_Merge(AlignList, i, alignment):
    if(i==0):
        #assign address to min TLB value, for now use 0
        StartAddress = 0 
    else:
        min_address = (AlignList.iloc[i]['Start Address'] + AlignList.iloc[i]['Size']+1)
        StartAddress = (min_address + alignment) - (min_address + alignment)%alignment
    return StartAddress     
    

In [899]:
# merge function
#b has addresses 
#a does not have addresses
def MergeAssignAddress(a, b, order):
    order = 'Start Address'
    c = pd.DataFrame(columns = memory.columns)
    #assign first to b
    if(len(b)>0):
        #add b[0] to the end of c
        c = c.append(b.loc[b.first_valid_index()])
        #remove b[0] from b
        b = b.drop(b.first_valid_index()) 
    while((len(a)!= 0) and (len(b) != 0)):        
        #assign a address to the earliest possible next address in c
       # a.loc[a.first_valid_index()][order] =  c.loc[-1][order] +  c.loc[-1]['Size']
        a.set_value(a.first_valid_index(), order, getNextStartAddress_Merge(c, len(c)-1, a.loc[a.first_valid_index()]['Alignment'] ))
        #a.loc[a.first_valid_index()][order] = getNextStartAddress_Merge(c, len(c)-1, a.loc[a.first_valid_index()]['Alignment'] )
        if(int(a.loc[a.first_valid_index()][order]) + int(a.loc[a.first_valid_index()]['Size']) > int(b.loc[b.first_valid_index()][order])):
            #add b[0] to the end of c
            c = c.append(b.loc[b.first_valid_index()])
            #remove b[0] from b
            b = b.drop(b.first_valid_index())   
        else:
            # add a[0] to the end of c
            c = c.append(a.loc[a.first_valid_index()])
            #rmeove a[0] from a
            a = a.drop(a.first_valid_index())

    while (len(a) != 0):     
        #assign a
        #assign a address to the earliest possible next address in c
        #a.loc[a.first_valid_index()][order] =  c.loc[-1][order] +  c.loc[-1]['Size']
        a.set_value(a.first_valid_index(),order, getNextStartAddress_Merge(c, len(c)-1, a.loc[a.first_valid_index()]['Alignment'] ))
        # add a[0] to the end of c
        c = c.append(a.loc[a.first_valid_index()])
        #rmeove a[0] from a
        a = a.drop(a.first_valid_index()) 
    while (len(b) != 0): 
        # add b[0] to the end of c
        c = c.append(b.loc[b.first_valid_index()])
        #rmeove a[0] from a
        b = b.drop(b.first_valid_index())  
    return c

In [900]:
def MergeSort_AssignAddress(mem):
    n = len(mem)
    if ( n == 1 ): return mem
    
    l1 = mem[0:math.floor(n/2)]
    l2 = mem[math.floor(n/2) : n]

    l1 = MergeSort_AssignAddress( l1, 'Start Address' )
    l2 = MergeSort_AssignAddress( l2, 'Start Address' )

    return MergeAssignAddress( l1, l2, 'Start Address' )

In [919]:
def getData(memory_filename,tlb_filename):
    ########## AVERAGE CASE IMPORT DATA #######################
    tlbs_average =  pd.read_csv(tlb_filename)
    tlbs_average = tlbs_average.dropna()
    memory_average = pd.read_csv(memory_filename)
    memory_average = memory_average.dropna()

    #format data 
    tlbs_average['Start Address'] = tlbs_average['Start Address'].apply(int, base=16)
    tlbs_average['Min Size'] = tlbs_average['Min Size'].apply(int, base=16)
    tlbs_average['Size In KB'] = tlbs_average['Size In KB'].apply(str)
    tlbs_average['Size In KB'] = tlbs_average['Size In KB'].apply(int, base=16)

    memory_average['Alignment'] = memory_average['Alignment'].apply(str)
    memory_average['Alignment'] = memory_average['Alignment'].apply(int, base=16)
    memory_average['Size'] = memory_average['Size'].apply(int, base=16)
    memory = memory_average
    ################## SORT TLBS ##################
    tlb = tlbs_average
    tlb = MergeSort_Backwards(tlb, 'Start Address')
    num_tlbs = len(tlb)
    ################# Divide Memory sections into TLBs ###############
    tlb_memlist = []
    temp = pd.DataFrame()
    for i in range(num_tlbs):
        temp = (memory_average.loc[memory_average['TLB'] == tlb.iloc[i]['Name']])
        tlb_memlist.append(temp)
        # memory_average.loc[memory_average['TLB'] == 'User_TLB_DDR_3_1']
    return tlbs_average, memory_average, tlb_memlist

START USING built in functions

In [912]:
import time
def main_merge(memory_filename,tlb_filename):
    total_time = 0
     #########################
    #IMPORT
    runtime = 0
    tlb_average = pd.DataFrame()
    memory_average = pd.DataFrame()
    tlb_memlist = []
    tlb_average, memory_average, tlb_memlist =  getData(memory_filename,tlb_filename)
    memory = memory_average
    tlb = tlb_average
    ###############################
    #assign memory 
    memory = tlb_memlist[0].copy()
    memory['Start Address'].values[:] = 0
    ###############################
    mem_map = pd.DataFrame(columns = memory.columns)
    
    #for each TLB, sort
    for i in range(len(tlb_memlist)):
        #get the TLB start address
        tlb_start = tlb_average.iloc[i]['Start Address']
        memory = tlb_memlist[i].copy()
        memory['Start Address'].values[:] = tlb_start#clear start addresses

        #create a list of aligment organized arrays
        df0 = pd.DataFrame(columns = memory.columns)
        df1 = pd.DataFrame(columns = memory.columns)
        df2 = pd.DataFrame(columns = memory.columns)
        df3 = pd.DataFrame(columns = memory.columns)
        df4 = pd.DataFrame(columns = memory.columns)
        df5 = pd.DataFrame(columns = memory.columns)
        df6 = pd.DataFrame(columns = memory.columns)

        AlignmentLists = [df0,df1,df2,df3,df4,df5,df6]

        #count the number of 
        #(0,4,16,64,256,1024,4096) 
        alignment_cnt = [0,0,0,0,0,0,0]
        for i in range(len(memory)):
            alignment_cnt[int(math.log(memory.iloc[i]['Alignment'],4))]+=1  
        #start = time.perf_counter()
        b, temp_time=MergeSortTest(memory)
       # stop = time.perf_counter()
        total_time = total_time + temp_time
        mem_map = mem_map.append(b)
    return total_time

In [913]:
import time
def main_quicksort(memory_filename, tlb_filename):
    total_time = 0
     #########################
    #IMPORT
    runtime = 0
    tlb_average = pd.DataFrame()
    memory_average = pd.DataFrame()
    tlb_memlist = []
    tlb_average, memory_average, tlb_memlist =  getData(memory_filename, tlb_filename)
    memory = memory_average
    tlb = tlb_average
    ###############################
    #assign memory 
    memory = tlb_memlist[0].copy()
    memory['Start Address'].values[:] = 0
    ###############################
    mem_map = pd.DataFrame(columns = memory.columns)
    
    #for each TLB, sort
    for i in range(len(tlb_memlist)):
        #get the TLB start address
        tlb_start = tlb_average.iloc[i]['Start Address']
        memory = tlb_memlist[i].copy()
        memory['Start Address'].values[:] = tlb_start#clear start addresses

        #create a list of aligment organized arrays
        df0 = pd.DataFrame(columns = memory.columns)
        df1 = pd.DataFrame(columns = memory.columns)
        df2 = pd.DataFrame(columns = memory.columns)
        df3 = pd.DataFrame(columns = memory.columns)
        df4 = pd.DataFrame(columns = memory.columns)
        df5 = pd.DataFrame(columns = memory.columns)
        df6 = pd.DataFrame(columns = memory.columns)

        AlignmentLists = [df0,df1,df2,df3,df4,df5,df6]

        #count the number of 
        #(0,4,16,64,256,1024,4096) 
        alignment_cnt = [0,0,0,0,0,0,0]
        for i in range(len(memory)):
            alignment_cnt[int(math.log(memory.iloc[i]['Alignment'],4))]+=1  
        b, temp_time=QuickSortTest(memory)
        total_time = total_time + temp_time
        mem_map = mem_map.append(b)
    return total_time

In [905]:
# at the top of the file, before other imports
import warnings

warnings.filterwarnings('ignore')

def MergeSortTest(memory):
    total_time = 0
    start  = time.perf_counter()
    #sort by alignment
    c = pd.DataFrame(columns = memory.columns)
    memory.sort_values(['Alignment'], axis = 0, ascending = False, inplace=True, kind='mergesort')
    c = memory.copy()
    

    #split into data frames by alignment
    first = 0
    for i in range (NUM_ALIGNMENTS):
        AlignmentLists[NUM_ALIGNMENTS-1-i] = c[first:first+alignment_cnt[NUM_ALIGNMENTS-1-i]]
        first = first + alignment_cnt[NUM_ALIGNMENTS-1-i]
     #sort by size
    #sort each alignment df by size in decending order
    order = 'Size'
    for i in range (NUM_ALIGNMENTS):
        if(len(AlignmentLists[i])>1):
            #AlignmentLists[i] = MergeSort(AlignmentLists[i], order)    
            AlignmentLists[i].sort_values(['Size'], axis = 0, ascending = False, inplace=True, kind='mergesort')  
    end = time.perf_counter()
    #assign start addresses starting from largest to smallest and merge
    for i in range (NUM_ALIGNMENTS):
        if (len(AlignmentLists[NUM_ALIGNMENTS-1-i])!=0):
            first_nonzero = i
            break
    for j in range(len(AlignmentLists[NUM_ALIGNMENTS-1-i])):
        AlignmentLists[NUM_ALIGNMENTS-1-i].set_value(AlignmentLists[NUM_ALIGNMENTS-1-i].index[j], 'Start Address',getNextStartAddress(AlignmentLists[NUM_ALIGNMENTS-1-i],j))
    
    #merge all
    b = AlignmentLists[NUM_ALIGNMENTS-1-first_nonzero]
    for j in range(first_nonzero+1,NUM_ALIGNMENTS):
        a = AlignmentLists[NUM_ALIGNMENTS-1-j-first_nonzero]
        b = MergeAssignAddress(a, b, 'Start Address')
    total_time = end-start
    return b, total_time

In [906]:
# at the top of the file, before other imports
import warnings

warnings.filterwarnings('ignore')

def QuickSortTest(memory):
    total_time = 0
    start = time.perf_counter()
    #sort by alignment
    c = pd.DataFrame(columns = memory.columns)
    memory.sort_values(['Alignment'], axis = 0, ascending = False, inplace=True, kind='quicksort')
    c = memory.copy()
    

    #split into data frames by alignment
    first = 0
    for i in range (NUM_ALIGNMENTS):
        AlignmentLists[NUM_ALIGNMENTS-1-i] = c[first:first+alignment_cnt[NUM_ALIGNMENTS-1-i]]
        first = first + alignment_cnt[NUM_ALIGNMENTS-1-i]
     #sort by size
    #sort each alignment df by size in decending order
    order = 'Size'
    for i in range (NUM_ALIGNMENTS):
        if(len(AlignmentLists[i])>1):
            #AlignmentLists[i] = MergeSort(AlignmentLists[i], order)    
            AlignmentLists[i].sort_values(['Size'], axis = 0, ascending = False, inplace=True, kind='quicksort')  
    end = time.perf_counter()
    #assign start addresses starting from largest to smallest and merge
    for i in range (NUM_ALIGNMENTS):
        if (len(AlignmentLists[NUM_ALIGNMENTS-1-i])!=0):
            first_nonzero = i
            break
    for j in range(len(AlignmentLists[NUM_ALIGNMENTS-1-i])):
        AlignmentLists[NUM_ALIGNMENTS-1-i].set_value(AlignmentLists[NUM_ALIGNMENTS-1-i].index[j], 'Start Address',getNextStartAddress(AlignmentLists[NUM_ALIGNMENTS-1-i],j))

    #merge all
    b = AlignmentLists[NUM_ALIGNMENTS-1-first_nonzero]
    for j in range(first_nonzero+1,NUM_ALIGNMENTS):
        a = AlignmentLists[NUM_ALIGNMENTS-1-j-first_nonzero]
        b = MergeAssignAddress(a, b, 'Start Address')
    total_time = end-start
    return b, total_time

In [922]:
#AVERAGE CASE
time_quicksort = main_quicksort('generated_memory_sections_average_case.csv', 'generated_tlbs_average_case.csv' )
print("Average Case: ",time_quicksort)

time_quicksort = main_merge('generated_memory_sections_worst_case.csv', 'generated_tlbs_worst_case.csv')
print("Worst Case: ", time_quicksort)

time_quicksort = main_merge('generated_memory_sections_best_case.csv', 'generated_tlbs_best_case.csv')
print("Best Case: ", time_quicksort)

Average Case:  27.788452600012533
Worst Case:  25.938282400020398
Best Case:  20.46467400016263


In [921]:
time_mergesort = main_merge('generated_memory_sections_average_case.csv', 'generated_tlbs_average_case.csv')
print("Average Case: ", time_mergesort)

time_mergesort = main_merge('generated_memory_sections_worst_case.csv', 'generated_tlbs_worst_case.csv')
print("Worst Case: ", time_mergesort)

time_mergesort = main_merge('generated_memory_sections_best_case.csv', 'generated_tlbs_best_case.csv')
print("Best Case: ", time_mergesort)

Average Case:  24.063392500393093
Worst Case:  23.573665499687195
Best Case:  24.58532800001558
